Simple Gen AI App Using Langchain

In [19]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
#Langsmith tracking
os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGCHAIN_API_KEY')
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ["LANGCHAIN_PROJECT"] = os.getenv('LANGCHAIN_PROJECT')

In [20]:
## Data InIngestion -- From website we need to scrape the data

from langchain_community.document_loaders import WebBaseLoader


In [22]:
loader = WebBaseLoader("https://docs.langchain.com/langsmith/home")

data = loader.load()
data


[Document(metadata={'source': 'https://docs.langchain.com/langsmith/home', 'title': 'LangSmith docs - Docs by LangChain', 'language': 'en'}, page_content="LangSmith docs - Docs by LangChainSkip to main contentWe've raised a $125M Series B to build the platform for agent engineering. Read more.Docs by LangChain home pageLangSmithSearch...⌘KGitHubTry LangSmithTry LangSmithSearch...NavigationLangSmith docsGet startedObservabilityEvaluationPrompt engineeringDeploymentAgent BuilderPlatform setupOverviewPlansCreate an account and API keyAccount administrationOverviewSet up a workspaceManage organizations using the APIManage billingSet up resource tagsUser managementReferenceLangSmith Python SDKLangSmith JS/TS SDKLangGraph Python SDKLangGraph JS/TS SDKLangSmith APIAPI reference for LangSmith DeploymentAdditional resourcesReleases & changelogsData managementScalability & resilienceAuthentication methodsFAQsRegions FAQPricing FAQLangSmith docsCopy pageCopy pageLangSmith provides tools for devel

In [ ]:
### Load Data --> Docs --> Divide our data into Chunks -->Vector --> Vector Embeddings --> Vector StoreDB

In [46]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = text_splitter.split_documents(data)
docs

[Document(metadata={'source': 'https://docs.langchain.com/langsmith/home', 'title': 'LangSmith docs - Docs by LangChain', 'language': 'en'}, page_content="LangSmith docs - Docs by LangChainSkip to main contentWe've raised a $125M Series B to build the platform for agent engineering. Read more.Docs by LangChain home pageLangSmithSearch...⌘KGitHubTry LangSmithTry LangSmithSearch...NavigationLangSmith docsGet startedObservabilityEvaluationPrompt engineeringDeploymentAgent BuilderPlatform setupOverviewPlansCreate an account and API keyAccount administrationOverviewSet up a workspaceManage organizations using the APIManage billingSet up resource tagsUser managementReferenceLangSmith Python SDKLangSmith JS/TS SDKLangGraph Python SDKLangGraph JS/TS SDKLangSmith APIAPI reference for LangSmith DeploymentAdditional resourcesReleases & changelogsData managementScalability & resilienceAuthentication methodsFAQsRegions FAQPricing FAQLangSmith docsCopy pageCopy pageLangSmith provides tools for devel

In [47]:
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()



In [48]:
from langchain_community.vectorstores import FAISS
vectorstore = FAISS.from_documents(docs, embeddings)

In [26]:
vectorstore

In [49]:
#### Querying the Vector Store
query="choose a quickstart to begin building with LangSmith"
results = vectorstore.similarity_search(query)
results[0].page_content

'Copy the key and save it securely.\nOnce your account and API key are ready, choose a quickstart to begin building with LangSmith:\nObservabilityGain visibility into every step your application takes to debug faster and improve reliability.Start tracingEvaluationMeasure and track quality over time to ensure your AI applications are consistent and trustworthy.Evaluate your appDeploymentDeploy your agents as LangGraph Servers, ready to scale in production.Deploy your agentsPlatform setupUse LangSmith in managed cloud, in a self-hosted environment, or hybrid to match your infrastructure and compliance needs.Choose how to set up LangSmithPrompt TestingIterate on prompts with built-in versioning and collaboration to ship improvements faster.Test your promptsStudioUse a visual interface to design, test, and refine applications end-to-end.Develop with Studio'

In [50]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o", temperature=0)

In [51]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.documents import Document


# ✅ 4. Define your prompt
prompt=ChatPromptTemplate.from_template(
    """
Answer the following question based only on the provided context:
<context>
{context}
</context>


"""
)

# ✅ 5. Combine documents manually (replacement for create_stuff_documents_chain)
def combine_docs(docs):
    """Concatenate text from all retrieved documents."""
    return "\n\n".join([d.page_content for d in docs])

# ✅ 6. Build the LCEL chain (Stuff → Prompt → LLM)
document_chain = (
    RunnablePassthrough.assign(context=lambda x: combine_docs(x["docs"]))
    | prompt
    | llm
)



# result = document_chain.invoke({
#     "input":"LangSmith has two usage limits: total traces and extended",
#     "docs":[Document(page_content="LangSmith has two usage limits: total traces and extended traces. These correspond to the two metrics we've been tracking on our usage graph. ")]
# })

docs = [
    Document(page_content="LangSmith has two usage limits: total traces and extended traces. These correspond to the two metrics we've been tracking on our usage graph.")
]

retriever=vectorstore.as_retriever()

# ✅ 6. Create LCEL chain (Retriever + Stuff + Prompt + LLM)
retrieval_chain = (
    {
        "context": retriever | (lambda docs: combine_docs(docs)),
        "question": RunnablePassthrough()
    }
    | prompt
    | llm
)



result = retrieval_chain.invoke("choose a quickstart to begin building with LangSmith")
print("\n📘 Final Answer:\n", result.content)


📘 Final Answer:
 What is LangSmith, and what are its key features?
